In [5]:
import pandas as pd

# Replace 'transactions.csv' with your actual file name if different
df = pd.read_csv('C:\\Users\\Owner\\Project\\transaction.csv')

# Display the first 5 rows to check the data
print(df.head())

# Check data types and basic info
print(df.info())


   step      type    amount     nameOrig  oldbalanceOrg  newbalanceOrig  \
0     1   PAYMENT   9839.64  C1231006815       170136.0       160296.36   
1     1   PAYMENT   1864.28  C1666544295        21249.0        19384.72   
2     1  TRANSFER    181.00  C1305486145          181.0            0.00   
3     1  CASH_OUT    181.00   C840083671          181.0            0.00   
4     1   PAYMENT  11668.14  C2048537720        41554.0        29885.86   

      nameDest  oldbalanceDest  newbalanceDest  isFraud  isFlaggedFraud  
0  M1979787155             0.0             0.0        0               0  
1  M2044282225             0.0             0.0        0               0  
2   C553264065             0.0             0.0        1               0  
3    C38997010         21182.0             0.0        1               0  
4  M1230701703             0.0             0.0        0               0  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6362620 entries, 0 to 6362619
Data columns (total 11 co

In [7]:
print(df.columns)
print(df.describe())
print(df.isnull().sum())

Index(['step', 'type', 'amount', 'nameOrig', 'oldbalanceOrg', 'newbalanceOrig',
       'nameDest', 'oldbalanceDest', 'newbalanceDest', 'isFraud',
       'isFlaggedFraud'],
      dtype='object')
               step        amount  oldbalanceOrg  newbalanceOrig  \
count  6.362620e+06  6.362620e+06   6.362620e+06    6.362620e+06   
mean   2.433972e+02  1.798619e+05   8.338831e+05    8.551137e+05   
std    1.423320e+02  6.038582e+05   2.888243e+06    2.924049e+06   
min    1.000000e+00  0.000000e+00   0.000000e+00    0.000000e+00   
25%    1.560000e+02  1.338957e+04   0.000000e+00    0.000000e+00   
50%    2.390000e+02  7.487194e+04   1.420800e+04    0.000000e+00   
75%    3.350000e+02  2.087215e+05   1.073152e+05    1.442584e+05   
max    7.430000e+02  9.244552e+07   5.958504e+07    4.958504e+07   

       oldbalanceDest  newbalanceDest       isFraud  isFlaggedFraud  
count    6.362620e+06    6.362620e+06  6.362620e+06    6.362620e+06  
mean     1.100702e+06    1.224996e+06  1.290820e-03  

In [13]:
print(type(df))

<class 'pandas.core.frame.DataFrame'>


In [15]:
# Count missing values in each column
missing_report = df.isnull().sum()
print("Missing Values per Column:\n", missing_report)


Missing Values per Column:
 step              0
type              0
amount            0
nameOrig          0
oldbalanceOrg     0
newbalanceOrig    0
nameDest          0
oldbalanceDest    0
newbalanceDest    0
isFraud           0
isFlaggedFraud    0
dtype: int64


In [17]:
# Check for duplicate rows
duplicates = df[df.duplicated()]
print(f"Number of duplicate rows: {duplicates.shape[0]}")

# Check for duplicate transaction IDs (if you have a 'transaction_id' column)
if 'transaction_id' in df.columns:
    duplicate_ids = df[df.duplicated(subset=['transaction_id'])]
    print(f"Number of duplicate transaction IDs: {duplicate_ids.shape[0]}")


Number of duplicate rows: 0


In [19]:
# Check for negative transaction amounts
if 'amount' in df.columns:
    negative_amounts = df[df['amount'] < 0]
    print(f"Number of negative amounts: {negative_amounts.shape[0]}")

# Check for future dates (if you have a 'date' column)
import datetime
if 'date' in df.columns:
    df['date'] = pd.to_datetime(df['date'], errors='coerce')
    future_dates = df[df['date'] > datetime.datetime.now()]
    print(f"Number of transactions with future dates: {future_dates.shape[0]}")


Number of negative amounts: 0


In [23]:
from sklearn.ensemble import IsolationForest

if 'amount' in df.columns:
    # Isolation Forest expects 2D input
    amounts = df[['amount']].dropna()
    clf = IsolationForest(contamination=0.01, random_state=42)
    df.loc[amounts.index, 'anomaly'] = clf.fit_predict(amounts)
    anomalies = df[df['anomaly'] == -1]
    print(f"Number of anomalies detected: {anomalies.shape[0]}")


Number of anomalies detected: 61419


In [25]:
summary = {
    'Missing Values': missing_report.sum(),
    'Duplicate Rows': duplicates.shape[0],
    'Negative Amounts': negative_amounts.shape[0] if 'amount' in df.columns else 0,
    'Future Dates': future_dates.shape[0] if 'date' in df.columns else 0,
    'Anomalies': anomalies.shape[0] if 'amount' in df.columns else 0
}
summary_df = pd.DataFrame(list(summary.items()), columns=['Issue', 'Count'])
print(summary_df)


              Issue  Count
0    Missing Values      0
1    Duplicate Rows      0
2  Negative Amounts      0
3      Future Dates      0
4         Anomalies  61419


In [27]:
summary_df.to_csv("C:\\Users\\Owner\\Project\\Financial data quality project\\report\\data_quality_summary.csv", index=False)
